In [ ]:
import tensorflow as tf
import numpy as np
import plotly.graph_objects as go

# Define the neural network model
class PINN(tf.keras.Model):
    def __init__(self, activation='tanh'):
        super(PINN, self).__init__()
        self.hidden = [
            tf.keras.layers.Dense(64, activation=activation),
            tf.keras.layers.Dense(64, activation=activation),
            tf.keras.layers.Dense(64, activation=activation),
            tf.keras.layers.Dense(1)
        ]

    def call(self, inputs):
        for layer in self.hidden:
            inputs = layer(inputs)
        return inputs

# Numerical approximation of the Caputo fractional derivative
def caputo_time_derivative(u, t, alpha):
    dt = t[1] - t[0]
    n = t.shape[0]
    term = 0
    for j in range(1, n):
        term += (u[:, j:j+1] - u[:, j-1:j]) / (dt**alpha) * (t[j:j+1] - t[j-1:j])**(1 - alpha)
    term = term[:, :-1]  # Correct shape to match
    return term / tf.exp(tf.math.lgamma(2 - alpha))

# Numerical approximation of the Riesz fractional derivative
def riesz_space_derivative(u, x, beta):
    dx = x[1] - x[0]
    n = x.shape[0]
    term = 0
    for j in range(1, n-1):
        term += (u[j+1:j+2, :] - 2 * u[j:j+1, :] + u[j-1:j, :]) / (dx**beta) * (x[j+1:j+2] - x[j-1:j])**(2 - beta)
    term = term[:-2, :]  # Correct shape to match
    return term / tf.exp(tf.math.lgamma(3 - beta))

# Loss function
def loss_function(model, x, t, alpha, beta, v, D):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([x, t])
        inputs = tf.concat([x, t], axis=1)
        u = model(inputs)
        u_t = caputo_time_derivative(u, t, alpha)
        u_x = tape.gradient(u, x)
        u_xx = riesz_space_derivative(u, x, beta)
    pde_loss = u_t + v * u_x - D * u_xx  # Align shapes
    return tf.reduce_mean(pde_loss**2)

# Training
model = PINN()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Define training data (example)
x = np.linspace(0, 1, 100).reshape(-1, 1).astype(np.float32)
t = np.linspace(0, 1, 100).reshape(-1, 1).astype(np.float32)
X, T = np.meshgrid(x, t)
X_flat = X.flatten().reshape(-1, 1)
T_flat = T.flatten().reshape(-1, 1)

X_flat_tensor = tf.convert_to_tensor(X_flat, dtype=tf.float32)
T_flat_tensor = tf.convert_to_tensor(T_flat, dtype=tf.float32)

alpha = 0.8  # example value
beta = 1.5  # example value
v = 1.0  # example value
D = 1.0  # example value

# Training loop
epochs = 1000
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        loss = loss_function(model, X_flat_tensor, T_flat_tensor, alpha, beta, v, D)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Plotting the result using Plotly
u_pred = model(tf.concat([X_flat_tensor, T_flat_tensor], axis=1)).numpy().reshape(100, 100)

fig = go.Figure(data=[
    go.Surface(z=u_pred, x=X, y=T, colorscale='Viridis')
])

fig.update_layout(
    title='Predicted Solution u(x,t)',
    scene=dict(
        xaxis_title='x',
        yaxis_title='t',
        zaxis_title='u(x,t)',
        bgcolor='white'
    ),
    autosize=False,
    width=700,
    height=700,
    margin=dict(l=65, r=50, b=65, t=90)
)

fig.show()


InvalidArgumentError: {{function_node __wrapped__Sub_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [10000,0] vs. [0,1] [Op:Sub] name: 

In [ ]:
pip install numfracpy

In [ ]:
import numfracpy as nfp

print(dir(nfp))


['Caputo_der1', 'Caputo_der2', 'FODE', 'GLDer1', 'GLDer2', 'LTInversion', 'MittagLeffler_one_fsum', 'MittagLeffler_two_fsum', 'Mittag_Leffler_one', 'Mittag_Leffler_three', 'Mittag_Leffler_two', 'OptimalParam_RB', 'OptimalParam_RU', 'RL_der1', 'RL_der2', 'RL_integral', 'SystemFODEs', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'epsilon', 'integrate', 'log_epsilon', 'math', 'np', 'special']


In [ ]:
import tensorflow as tf
import numpy as np
import plotly.graph_objects as go
from scipy.special import gamma

# Define the neural network model
class PINN(tf.keras.Model):
    def __init__(self, activation='tanh'):
        super(PINN, self).__init__()
        self.hidden = [
            tf.keras.layers.Dense(64, activation=activation),
            tf.keras.layers.Dense(64, activation=activation),
            tf.keras.layers.Dense(64, activation=activation),
            tf.keras.layers.Dense(1)
        ]

    def call(self, inputs):
        for layer in self.hidden:
            inputs = layer(inputs)
        return inputs

# Numerical approximation of the Caputo fractional derivative
def caputo_time_derivative(u, t, alpha):
    dt = t[1, 0] - t[0, 0]
    u_np = u.numpy()
    u_t = np.zeros_like(u_np)
    for i in range(1, u_np.shape[0]):
        sum_term = 0
        for k in range(i):
            sum_term += (u_np[k] - u_np[k - 1]) / ((t[i] - t[k])**(1 + alpha))
        u_t[i] = dt**alpha / gamma(2 - alpha) * sum_term
    return tf.convert_to_tensor(u_t, dtype=tf.float32)

# Numerical approximation of the Riesz fractional derivative
def riesz_space_derivative(u, x, beta):
    dx = x[1, 0] - x[0, 0]
    u_np = u.numpy()
    u_xx = np.zeros_like(u_np)
    for i in range(1, u_np.shape[0] - 1):
        sum_term = 0
        for k in range(1, u_np.shape[1] - 1):
            sum_term += (u_np[i, k + 1] - 2 * u_np[i, k] + u_np[i, k - 1]) / ((x[i] - x[k])**(1 + beta))
        u_xx[i] = dx**beta / gamma(2 - beta) * sum_term
    return tf.convert_to_tensor(u_xx, dtype=tf.float32)

# Loss function
def loss_function(model, x, t, alpha, beta, v, D):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([x, t])
        inputs = tf.concat([x, t], axis=1)
        u = model(inputs)
        u_t = caputo_time_derivative(u, t, alpha)
        u_x = tape.gradient(u, x)
        u_xx = riesz_space_derivative(u, x, beta)
    pde_loss = u_t + v * u_x - D * u_xx
    return tf.reduce_mean(pde_loss**2)

# Training
model = PINN()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Define training data (example)
x = np.linspace(0, 1, 100).reshape(-1, 1).astype(np.float32)
t = np.linspace(0, 1, 100).reshape(-1, 1).astype(np.float32)
X, T = np.meshgrid(x, t)
X_flat = X.flatten().reshape(-1, 1)
T_flat = T.flatten().reshape(-1, 1)

X_flat_tensor = tf.convert_to_tensor(X_flat, dtype=tf.float32)
T_flat_tensor = tf.convert_to_tensor(T_flat, dtype=tf.float32)

alpha = 0.8  # example value
beta = 1.5  # example value
v = 1.0  # example value
D = 1.0  # example value

# Training loop
epochs = 1000
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        loss = loss_function(model, X_flat_tensor, T_flat_tensor, alpha, beta, v, D)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Plotting the result using Plotly
u_pred = model(tf.concat([X_flat_tensor, T_flat_tensor], axis=1)).numpy().reshape(100, 100)

fig = go.Figure(data=[
    go.Surface(z=u_pred, x=X, y=T, colorscale='Viridis')
])

fig.update_layout(
    title='Predicted Solution u(x,t)',
    scene=dict(
        xaxis_title='x',
        yaxis_title='t',
        zaxis_title='u(x,t)',
        bgcolor='white'
    ),
    autosize=False,
    width=700,
    height=700,
    margin=dict(l=65, r=50, b=65, t=90)
)

fig.show()


In [ ]:
import tensorflow as tf
import numpy as np
import plotly.graph_objects as go
from scipy.special import gamma

# Define the neural network model
class PINN(tf.keras.Model):
    def __init__(self, activation='tanh'):
        super(PINN, self).__init__()
        self.hidden = [
            tf.keras.layers.Dense(64, activation=activation),
            tf.keras.layers.Dense(64, activation=activation),
            tf.keras.layers.Dense(64, activation=activation),
            tf.keras.layers.Dense(1)
        ]

    def call(self, inputs):
        for layer in self.hidden:
            inputs = layer(inputs)
        return inputs

def caputo_time_derivative(u, t, alpha):
    dt = t[1, 0] - t[0, 0]
    u_flat = tf.reshape(u, [-1])
    t_flat = tf.reshape(t, [-1])
    u_t = tf.zeros_like(u_flat)

    for i in range(1, len(u_flat)):
        t_diff = (t_flat[i] - t_flat[:i])**(1 + alpha)
        u_diff = (u_flat[1:i+1] - u_flat[:i])
        u_t = u_t + tf.concat([tf.zeros(i), tf.reduce_sum(u_diff / t_diff) * tf.ones(1), tf.zeros(len(u_flat) - i - 1)], axis=0)

    u_t = dt**alpha / gamma(2 - alpha) * u_t
    return tf.reshape(u_t, [-1, 1])


def riesz_space_derivative(u, x, beta):
    dx = x[1, 0] - x[0, 0]
    u_flat = tf.reshape(u, [-1])
    x_flat = tf.reshape(x, [-1])
    u_xx = tf.zeros_like(u_flat)

    for i in range(1, len(u_flat) - 1):
        x_diff = (x_flat[i] - x_flat[:i])**(1 + beta)
        u_diff = (u_flat[i+1:i+2] - 2 * u_flat[i:i+1] + u_flat[i-1:i])
        u_xx = u_xx + tf.concat([tf.zeros(i), tf.reduce_sum(u_diff / x_diff) * tf.ones(1), tf.zeros(len(u_flat) - i - 1)], axis=0)

    u_xx = dx**beta / gamma(2 - beta) * u_xx
    return tf.reshape(u_xx, [-1, 1])


# Loss function
def loss_function(model, x, t, alpha, beta, v, D):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([x, t])
        inputs = tf.concat([x, t], axis=1)
        u = model(inputs)
        u_t = caputo_time_derivative(u, t, alpha)
        u_x = tape.gradient(u, x)
        u_xx = riesz_space_derivative(u, x, beta)
    pde_loss = u_t + v * u_x - D * u_xx
    return tf.reduce_mean(pde_loss**2)

# Training
model = PINN()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Define training data (example)
x = np.linspace(0, 1, 100).reshape(-1, 1).astype(np.float32)
t = np.linspace(0, 1, 100).reshape(-1, 1).astype(np.float32)
X, T = np.meshgrid(x, t)
X_flat = X.flatten().reshape(-1, 1)
T_flat = T.flatten().reshape(-1, 1)

X_flat_tensor = tf.convert_to_tensor(X_flat, dtype=tf.float32)
T_flat_tensor = tf.convert_to_tensor(T_flat, dtype=tf.float32)

alpha = 0.8  # example value
beta = 1.5  # example value
v = 1.0  # example value
D = 1.0  # example value

# Training loop
epochs = 1000
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        loss = loss_function(model, X_flat_tensor, T_flat_tensor, alpha, beta, v, D)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Plotting the result using Plotly
u_pred = model(tf.concat([X_flat_tensor, T_flat_tensor], axis=1)).numpy().reshape(100, 100)

fig = go.Figure(data=[
    go.Surface(z=u_pred, x=X, y=T, colorscale='Viridis')
])

fig.update_layout(
    title='Predicted Solution u(x,t)',
    scene=dict(
        xaxis_title='x',
        yaxis_title='t',
        zaxis_title='u(x,t)',
        bgcolor='white'
    ),
    autosize=False,
    width=700,
    height=700,
    margin=dict(l=65, r=50, b=65, t=90)
)

fig.show()


Epoch 0, Loss: nan


In [ ]:
import tensorflow as tf
import numpy as np
import plotly.graph_objects as go
from scipy.special import gamma

# Define the neural network model
class PINN(tf.keras.Model):
    def __init__(self, activation='tanh'):
        super(PINN, self).__init__()
        self.hidden = [
            tf.keras.layers.Dense(64, activation=activation),
            tf.keras.layers.Dense(64, activation=activation),
            tf.keras.layers.Dense(64, activation=activation),
            tf.keras.layers.Dense(1)
        ]

    def call(self, inputs):
        for layer in self.hidden:
            inputs = layer(inputs)
        return inputs

# Numerical approximation of the Caputo fractional derivative using TensorFlow operations
def caputo_time_derivative(u, t, alpha):
    dt = t[1, 0] - t[0, 0]
    u_flat = tf.reshape(u, [-1])
    t_flat = tf.reshape(t, [-1])
    u_t = tf.zeros_like(u_flat)

    for i in range(1, len(u_flat)):
        t_diff = (t_flat[i] - t_flat[:i])**(1 + alpha)
        u_diff = (u_flat[1:i+1] - u_flat[:i])
        u_t = u_t + tf.concat([tf.zeros(i), tf.reduce_sum(u_diff / t_diff) * tf.ones(1), tf.zeros(len(u_flat) - i - 1)], axis=0)

    u_t = dt**alpha / gamma(2 - alpha) * u_t
    return tf.reshape(u_t, [-1, 1])

# Numerical approximation of the Riesz fractional derivative using TensorFlow operations
def riesz_space_derivative(u, x, beta):
    dx = x[1, 0] - x[0, 0]
    u_flat = tf.reshape(u, [-1])
    x_flat = tf.reshape(x, [-1])
    u_xx = tf.zeros_like(u_flat)

    for i in range(1, len(u_flat) - 1):
        x_diff = (x_flat[i] - x_flat[:i])**(1 + beta)
        u_diff = (u_flat[i+1:i+2] - 2 * u_flat[i:i+1] + u_flat[i-1:i])
        u_xx = u_xx + tf.concat([tf.zeros(i), tf.reduce_sum(u_diff / x_diff) * tf.ones(1), tf.zeros(len(u_flat) - i - 1)], axis=0)

    u_xx = dx**beta / gamma(2 - beta) * u_xx
    return tf.reshape(u_xx, [-1, 1])

# Loss function
def loss_function(model, x, t, alpha, beta, v, D):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([x, t])
        inputs = tf.concat([x, t], axis=1)
        u = model(inputs)
        u_t = caputo_time_derivative(u, t, alpha)
        u_x = tape.gradient(u, x)
        u_xx = riesz_space_derivative(u, x, beta)
    pde_loss = u_t + v * u_x - D * u_xx
    return tf.reduce_mean(pde_loss**2)

# Training
model = PINN()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Define training data (example)
x = np.linspace(0, 1, 10).reshape(-1, 1).astype(np.float32)
t = np.linspace(0, 1, 10).reshape(-1, 1).astype(np.float32)
X, T = np.meshgrid(x, t)
X_flat = X.flatten().reshape(-1, 1)
T_flat = T.flatten().reshape(-1, 1)

X_flat_tensor = tf.convert_to_tensor(X_flat, dtype=tf.float32)
T_flat_tensor = tf.convert_to_tensor(T_flat, dtype=tf.float32)

alpha = 0.8  # example value
beta = 1.5  # example value
v = 1.0  # example value
D = 1.0  # example value

# Training loop
epochs = 1000
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        loss = loss_function(model, X_flat_tensor, T_flat_tensor, alpha, beta, v, D)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Plotting the result using Plotly
# Predicting u_pred
predictions = model(tf.concat([X_flat_tensor, T_flat_tensor], axis=1))
u_pred = predictions.numpy().reshape(X.shape)  # Reshape to match X (100x100 grid)

# Plotting the result using Plotly
fig = go.Figure(data=[
    go.Surface(z=u_pred, x=X, y=T, colorscale='Viridis')
])

fig.update_layout(
    title='Predicted Solution u(x,t)',
    scene=dict(
        xaxis_title='x',
        yaxis_title='t',
        zaxis_title='u(x,t)',
        bgcolor='white'
    ),
    autosize=False,
    width=700,
    height=700,
    margin=dict(l=65, r=50, b=65, t=90)
)

fig.show()



Epoch 0, Loss: nan
Epoch 100, Loss: nan
Epoch 200, Loss: nan
Epoch 300, Loss: nan
Epoch 400, Loss: nan
Epoch 500, Loss: nan
Epoch 600, Loss: nan
Epoch 700, Loss: nan
Epoch 800, Loss: nan
Epoch 900, Loss: nan


In [3]:
import tensorflow as tf
import numpy as np
import plotly.graph_objects as go
from scipy.special import gamma

# Define the neural network model
class PINN(tf.keras.Model):
    def __init__(self, activation='tanh'):
        super(PINN, self).__init__()
        self.hidden = [
            tf.keras.layers.Dense(64, activation=activation),
            tf.keras.layers.Dense(64, activation=activation),
            tf.keras.layers.Dense(64, activation=activation),
            tf.keras.layers.Dense(1)
        ]

    def call(self, inputs):
        for layer in self.hidden:
            inputs = layer(inputs)
        return inputs

# Numerical approximation of the Caputo fractional derivative using TensorFlow operations
def caputo_time_derivative(u, t, alpha):
    dt = t[1] - t[0]
    u_flat = tf.reshape(u, [-1])
    t_flat = tf.reshape(t, [-1])
    u_t = tf.zeros_like(u_flat)

    for i in range(1, len(u_flat)):
        t_diff = (t_flat[i] - t_flat[:i])**(1 + alpha)
        u_diff = (u_flat[1:i+1] - u_flat[:i])
        u_t = u_t + tf.concat([tf.zeros(i), tf.reduce_sum(u_diff / t_diff) * tf.ones(1), tf.zeros(len(u_flat) - i - 1)], axis=0)

    u_t = dt**alpha / gamma(2 - alpha) * u_t
    return tf.reshape(u_t, [-1, 1])

# Numerical approximation of the Riesz fractional derivative using TensorFlow operations
def riesz_space_derivative(u, x, beta):
    dx = x[1] - x[0]
    u_flat = tf.reshape(u, [-1])
    x_flat = tf.reshape(x, [-1])
    u_xx = tf.zeros_like(u_flat)

    for i in range(1, len(u_flat) - 1):
        x_diff = (x_flat[i] - x_flat[:i])**(1 + beta)
        u_diff = (u_flat[i+1:i+2] - 2 * u_flat[i:i+1] + u_flat[i-1:i])
        u_xx = u_xx + tf.concat([tf.zeros(i), tf.reduce_sum(u_diff / x_diff) * tf.ones(1), tf.zeros(len(u_flat) - i - 1)], axis=0)

    u_xx = dx**beta / gamma(2 - beta) * u_xx
    return tf.reshape(u_xx, [-1, 1])

# Loss function
def loss_function(model, x, t, alpha, beta, v, D):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x)
        tape.watch(t)
        inputs = tf.concat([x, t], axis=1)
        u = model(inputs)
        u_x = tape.gradient(u, x)
    u_t = caputo_time_derivative(u, t, alpha)
    u_xx = riesz_space_derivative(u, x, beta)
    pde_loss = u_t + v * u_x - D * u_xx
    return tf.reduce_mean(pde_loss**2)

# Training
model = PINN()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Define training data
x = np.linspace(0, 1, 10).reshape(-1, 1).astype(np.float32)
t = np.linspace(1, 2, 10).reshape(-1, 1).astype(np.float32)
X, T = np.meshgrid(x, t)
X_flat = X.flatten().reshape(-1, 1)
T_flat = T.flatten().reshape(-1, 1)

X_flat_tensor = tf.convert_to_tensor(X_flat, dtype=tf.float32)
T_flat_tensor = tf.convert_to_tensor(T_flat, dtype=tf.float32)

# Generate initial condition u(x,0) = sin(pi * x)
initial_condition = np.sin(np.pi * x).astype(np.float32)

# Training loop
alpha = 0.8  # example value
beta = 1.2  # example value
v = 1.0  # example value
D = 1.0  # example value

epochs = 100
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        loss = loss_function(model, X_flat_tensor, T_flat_tensor, alpha, beta, v, D)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Predicting u_pred
predictions = model(tf.concat([X_flat_tensor, T_flat_tensor], axis=1))
u_pred = predictions.numpy().reshape(X.shape)  # Reshape to match X (50x50 grid)

# Plotting the result using Plotly
fig = go.Figure(data=[
    go.Surface(z=u_pred, x=X, y=T, colorscale='Viridis')
])

fig.update_layout(
    title='Predicted Solution u(x,t)',
    scene=dict(
        xaxis_title='x',
        yaxis_title='t',
        zaxis_title='u(x,t)',
        bgcolor='white'
    ),
    autosize=False,
    width=700,
    height=700,
    margin=dict(l=65, r=50, b=65, t=90)
)

fig.show()


Epoch 0, Loss: nan


In [2]:
import tensorflow as tf
import numpy as np
import plotly.graph_objects as go

# Define the neural network model
class PINN(tf.keras.Model):
    def __init__(self, activation='tanh'):
        super(PINN, self).__init__()
        self.hidden = [
            tf.keras.layers.Dense(64, activation=activation),
            tf.keras.layers.Dense(64, activation=activation),
            tf.keras.layers.Dense(64, activation=activation),
            tf.keras.layers.Dense(1)
        ]

    def call(self, inputs):
        for layer in self.hidden:
            inputs = layer(inputs)
        return inputs

# Loss function
def loss_function(model, x, t, v, D):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x)
        tape.watch(t)
        inputs = tf.concat([x, t], axis=1)
        u = model(inputs)
        u_x = tape.gradient(u, x)
        u_xx = tape.gradient(u_x, x)
        u_t = tape.gradient(u, t)
    pde_loss = u_t + v * u_x - D * u_xx
    return tf.reduce_mean(tf.square(pde_loss))

# Training
model = PINN()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Define training data
x = np.linspace(0, 1, 50).reshape(-1, 1).astype(np.float32)
t = np.linspace(0, 1, 50).reshape(-1, 1).astype(np.float32)
X, T = np.meshgrid(x, t)
X_flat = X.flatten().reshape(-1, 1)
T_flat = T.flatten().reshape(-1, 1)

X_flat_tensor = tf.convert_to_tensor(X_flat, dtype=tf.float32)
T_flat_tensor = tf.convert_to_tensor(T_flat, dtype=tf.float32)

# Generate initial condition u(x,0) = sin(pi * x)
initial_condition = np.sin(np.pi * x).astype(np.float32)

# Training loop
v = 1.0  # example value
D = 1.0  # example value

epochs = 1000
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        loss = loss_function(model, X_flat_tensor, T_flat_tensor, v, D)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Predicting u_pred
predictions = model(tf.concat([X_flat_tensor, T_flat_tensor], axis=1))
u_pred = predictions.numpy().reshape(X.shape)  # Reshape to match X (50x50 grid)

# Plotting the result using Plotly
fig = go.Figure(data=[
    go.Surface(z=u_pred, x=X, y=T, colorscale='Viridis')
])

fig.update_layout(
    title='Predicted Solution u(x,t)',
    scene=dict(
        xaxis_title='x',
        yaxis_title='t',
        zaxis_title='u(x,t)',
        bgcolor='white'
    ),
    autosize=False,
    width=700,
    height=700,
    margin=dict(l=65, r=50, b=65, t=90)
)

fig.show()


Epoch 0, Loss: 0.001163708744570613


Epoch 100, Loss: 5.352487278287299e-06


Epoch 200, Loss: 2.2015210561221465e-06


Epoch 300, Loss: 8.981759833659453e-07


Epoch 400, Loss: 3.6880527432003873e-07


Epoch 500, Loss: 1.9561746000817948e-07


Epoch 600, Loss: 1.4493359401512862e-07


Epoch 700, Loss: 1.2747661060075188e-07


Epoch 800, Loss: 1.1763677321141586e-07


Epoch 900, Loss: 1.0968921770881934e-07


In [9]:
import tensorflow as tf
import numpy as np
import plotly.graph_objects as go
from scipy.special import gamma

# Define the neural network model
class PINN(tf.keras.Model):
    def __init__(self, activation='tanh'):
        super(PINN, self).__init__()
        self.hidden = [
            tf.keras.layers.Dense(64, activation=activation, kernel_initializer='he_normal'),
            tf.keras.layers.Dense(64, activation=activation, kernel_initializer='he_normal'),
            tf.keras.layers.Dense(64, activation=activation, kernel_initializer='he_normal'),
            tf.keras.layers.Dense(1, kernel_initializer='he_normal')
        ]

    def call(self, inputs):
        for layer in self.hidden:
            inputs = layer(inputs)
        return inputs

# Numerical approximation of the Caputo fractional derivative using TensorFlow operations
def caputo_time_derivative(u, t, alpha):
    dt = t[1] - t[0]
    u_flat = tf.reshape(u, [-1])
    t_flat = tf.reshape(t, [-1])
    u_t = tf.zeros_like(u_flat)

    for i in range(1, len(u_flat)):
        t_diff = (t_flat[i] - t_flat[:i] + 1e-10)**(1 + alpha)  # Adding epsilon
        u_diff = (u_flat[1:i+1] - u_flat[:i])
        u_t = u_t + tf.concat([tf.zeros(i), tf.reduce_sum(u_diff / t_diff) * tf.ones(1), tf.zeros(len(u_flat) - i - 1)], axis=0)

    u_t = dt**alpha / gamma(2 - alpha) * u_t
    return tf.reshape(u_t, [-1, 1])

# Numerical approximation of the Riesz fractional derivative using TensorFlow operations
def riesz_space_derivative(u, x, beta):
    dx = x[1] - x[0]
    u_flat = tf.reshape(u, [-1])
    x_flat = tf.reshape(x, [-1])
    u_xx = tf.zeros_like(u_flat)

    for i in range(1, len(u_flat) - 1):
        x_diff = (x_flat[i] - x_flat[:i] + 1e-10)**(1 + beta)  # Adding epsilon
        u_diff = (u_flat[i+1:i+2] - 2 * u_flat[i:i+1] + u_flat[i-1:i])
        u_xx = u_xx + tf.concat([tf.zeros(i), tf.reduce_sum(u_diff / x_diff) * tf.ones(1), tf.zeros(len(u_flat) - i - 1)], axis=0)

    u_xx = dx**beta / gamma(2 - beta) * u_xx
    return tf.reshape(u_xx, [-1, 1])

# Loss function
def loss_function(model, x, t, alpha, beta, v, D):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x)
        tape.watch(t)
        inputs = tf.concat([x, t], axis=1)
        u = model(inputs)
        u_x = tape.gradient(u, x)
    u_t = caputo_time_derivative(u, t, alpha)
    u_xx = riesz_space_derivative(u, x, beta)
    pde_loss = u_t + v * u_x - D * u_xx

    print("Intermediate values:")
    print("u:", u.numpy())
    print("u_x:", u_x.numpy())
    print("u_t:", u_t.numpy())
    print("u_xx:", u_xx.numpy())
    print("pde_loss:", pde_loss.numpy())

    return tf.reduce_mean(pde_loss**2)

# Training
model = PINN()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)  # Reduced learning rate

# Define training data
x = np.linspace(0, 1, 10).reshape(-1, 1).astype(np.float32)
t = np.linspace(0, 1, 10).reshape(-1, 1).astype(np.float32)
X, T = np.meshgrid(x, t)
X_flat = X.flatten().reshape(-1, 1)
T_flat = T.flatten().reshape(-1, 1)

X_flat_tensor = tf.convert_to_tensor(X_flat, dtype=tf.float32)
T_flat_tensor = tf.convert_to_tensor(T_flat, dtype=tf.float32)

# Generate initial condition u(x,0) = sin(pi * x)
initial_condition = np.sin(np.pi * x).astype(np.float32)

# Training loop
alpha = 0.8  # example value
beta = 1.2  # example value
v = 1.0  # example value
D = 1.0  # example value

epochs = 100
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        loss = loss_function(model, X_flat_tensor, T_flat_tensor, alpha, beta, v, D)
    gradients = tape.gradient(loss, model.trainable_variables)
    gradients = [tf.clip_by_value(g, -1.0, 1.0) for g in gradients]  # Apply gradient clipping
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Predicting u_pred
predictions = model(tf.concat([X_flat_tensor, T_flat_tensor], axis=1))
u_pred = predictions.numpy().reshape(X.shape)  # Reshape to match X (10x10 grid)

# Plotting the result using Plotly
fig = go.Figure(data=[
    go.Surface(z=u_pred, x=X, y=T, colorscale='Viridis')
])

fig.update_layout(
    title='Predicted Solution u(x,t)',
    scene=dict(
        xaxis_title='x',
        yaxis_title='t',
        zaxis_title='u(x,t)',
        bgcolor='white'
    ),
    autosize=False,
    width=700,
    height=700,
    margin=dict(l=65, r=50, b=65, t=90)
)

fig.show()


Streaming output truncated to the last 5000 lines.
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
u_x: [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan

In [15]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from scipy import special

# Disable TensorFlow eager execution for performance
tf.compat.v1.disable_eager_execution()


In [16]:
class PINN(tf.keras.Model):
    def __init__(self):
        super(PINN, self).__init__()
        self.dense1 = keras.layers.Dense(50, activation='tanh', input_dim=2)  # Input layer
        self.dense2 = keras.layers.Dense(50, activation='tanh')
        self.dense3 = keras.layers.Dense(1, activation=None)  # Output layer

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        u = self.dense3(x)
        return u


In [17]:
def fractional_derivative(u, x, alpha):
    # Function to compute fractional derivative of u with respect to x of order alpha
    dx = x[1] - x[0]
    N = len(x)
    D = np.zeros((N, N))

    for i in range(N):
        for j in range(N):
            if i != j:
                D[i, j] = (x[i] - x[j])**(-alpha) * special.gamma(alpha + 1) / (special.gamma(1 - alpha) * dx**alpha)
            else:
                D[i, j] = -np.pi / (2 * dx**(1 + alpha)) * special.gamma(-alpha - 1)

    return np.dot(D, u)


def physics_loss(model, x_phys, t_phys, x_col, t_col, alpha, beta, v, D):
    # Function to compute the physics-informed loss

    def advection_diffusion_eq(x, t):
        # Define the advection-diffusion equation
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x)
            tape.watch(t)
            u = model(tf.stack([x, t], axis=1))
            du_dx = tape.gradient(u, x)
            du_dt = tape.gradient(u, t)

        # Compute fractional derivatives
        du_dx_alpha = fractional_derivative(du_dx.numpy(), x.numpy(), alpha)
        du_dt_alpha = fractional_derivative(du_dt.numpy(), t.numpy(), alpha)

        # Advection-diffusion equation residual
        residual = du_dt_alpha + v * du_dx_alpha - D * tf.gradients(tf.gradients(u, x), x)

        return residual

    loss = tf.reduce_mean(tf.square(advection_diffusion_eq(x_col, t_col)))

    return loss


In [18]:
def fractional_derivative(u, x, alpha):
    # Function to compute fractional derivative of u with respect to x of order alpha
    dx = x[1] - x[0]
    N = len(x)
    D = np.zeros((N, N))

    for i in range(N):
        for j in range(N):
            if i != j:
                D[i, j] = (x[i] - x[j])**(-alpha) * special.gamma(alpha + 1) / (special.gamma(1 - alpha) * dx**alpha)
            else:
                D[i, j] = -np.pi / (2 * dx**(1 + alpha)) * special.gamma(-alpha - 1)

    return np.dot(D, u)


def physics_loss(model, x_phys, t_phys, x_col, t_col, alpha, beta, v, D):
    # Function to compute the physics-informed loss

    def advection_diffusion_eq(x, t):
        # Define the advection-diffusion equation
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x)
            tape.watch(t)
            u = model(tf.stack([x, t], axis=1))
            du_dx = tape.gradient(u, x)
            du_dt = tape.gradient(u, t)

        # Compute fractional derivatives
        du_dx_alpha = fractional_derivative(du_dx.numpy(), x.numpy(), alpha)
        du_dt_alpha = fractional_derivative(du_dt.numpy(), t.numpy(), alpha)

        # Advection-diffusion equation residual
        residual = du_dt_alpha + v * du_dx_alpha - D * tf.gradients(tf.gradients(u, x), x)

        return residual

    loss = tf.reduce_mean(tf.square(advection_diffusion_eq(x_col, t_col)))

    return loss


In [19]:
def boundary_conditions(model, x_initial, u_initial, x_boundary, u_boundary):
    # Function to enforce initial and boundary conditions
    loss_init = tf.reduce_mean(tf.square(model(x_initial) - u_initial))
    loss_boundary = tf.reduce_mean(tf.square(model(x_boundary) - u_boundary))

    return loss_init + loss_boundary


In [22]:
def train_PINN(model, x_initial, u_initial, x_boundary, u_boundary, x_phys, t_phys, x_col, t_col, alpha, beta, v, D, epochs):
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            loss_phys = physics_loss(model, x_phys, t_phys, x_col, t_col, alpha, beta, v, D)
            loss_bc = boundary_conditions(model, x_initial, u_initial, x_boundary, u_boundary)
            total_loss = loss_phys + loss_bc

        grads = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 100 == 0:
            print(f'Epoch {epoch+1}, Total Loss: {total_loss.numpy()}, Physics Loss: {loss_phys.numpy()}, BC Loss: {loss_bc.numpy()}')

    print('Training completed.')


In [21]:
# Example usage
# Define your initial and boundary conditions, physical parameters, etc.

# Initialize PINN model
model = PINN()

# Define your training data (initial conditions, boundary conditions, collocation points, etc.)

# Train the PINN
train_PINN(model, x_initial, u_initial, x_boundary, u_boundary, x_phys, t_phys, x_col, t_col, alpha, beta, v, D, epochs=1000)


NameError: name 'x_initial' is not defined

In [33]:
import numpy as np
import tensorflow as tf # Import tensorflow at the beginning
from tensorflow import keras
from scipy import special
import plotly.graph_objs as go
import plotly.io as pio

# Enable eager execution
# tf.compat.v1.enable_eager_execution()

# Define the PINN model class
class PINN(tf.keras.Model):
    def __init__(self):
        super(PINN, self).__init__()
        self.dense1 = keras.layers.Dense(50, activation='tanh', input_dim=2)  # Input layer
        self.dense2 = keras.layers.Dense(50, activation='tanh')
        self.dense3 = keras.layers.Dense(1, activation=None)  # Output layer

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        u = self.dense3(x)
        return u

def fractional_derivative(u, x, alpha):
    dx = x[1] - x[0]
    N = x.shape[0]
    D = np.zeros((N, N))

    for i in range(N):
        for j in range(N):
            if i != j:
                D[i, j] = (x[i] - x[j])**(-alpha) * special.gamma(alpha + 1) / (special.gamma(1 - alpha) * dx**alpha)
            else:
                # Convert dx to a NumPy array before using it in calculations
                D[i, j] = -np.pi / (2 * dx.numpy()**(1 + alpha)) * special.gamma(-alpha - 1)

    return np.dot(D, u)

def physics_loss(model, x_phys, t_phys, x_col, t_col, alpha, beta, v, D):
    def advection_diffusion_eq(x, t):
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x)
            tape.watch(t)
            u = model(tf.stack([x, t], axis=1))
            du_dx = tape.gradient(u, x)
            du_dt = tape.gradient(u, t)

        # Convert TensorFlow tensors to NumPy arrays before passing them to fractional_derivative
        du_dx_alpha = fractional_derivative(du_dx.numpy(), x.numpy(), alpha)
        du_dt_alpha = fractional_derivative(du_dt.numpy(), t.numpy(), alpha)

        residual = du_dt_alpha + v * du_dx_alpha - D * tf.gradients(tf.gradients(u, x), x)

        return residual

    loss = tf.reduce_mean(tf.square(advection_diffusion_eq(x_col, t_col)))

    return loss

def boundary_conditions(model, x_initial, u_initial, x_boundary, u_boundary):
    x_boundary_tf = tf.constant(x_boundary, dtype=tf.float32)
    u_boundary_tf = tf.constant(u_boundary, dtype=tf.float32)

    loss_init = tf.reduce_mean(tf.square(model(x_initial) - u_initial))
    loss_boundary = tf.reduce_mean(tf.square(model(x_boundary_tf) - u_boundary_tf))

    return loss_init + loss_boundary

def train_PINN(model, x_initial, u_initial, x_boundary, u_boundary, x_phys, t_phys, x_col, t_col, alpha, beta, v, D, epochs):
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    # Convert numpy arrays to TensorFlow tensors
    x_initial_tf = tf.constant(x_initial, dtype=tf.float32)
    u_initial_tf = tf.constant(u_initial, dtype=tf.float32)
    x_boundary_tf = tf.constant(x_boundary, dtype=tf.float32)
    u_boundary_tf = tf.constant(u_boundary, dtype=tf.float32)
    x_phys_tf = tf.constant(x_phys, dtype=tf.float32)
    t_phys_tf = tf.constant(t_phys, dtype=tf.float32)
    x_col_tf = tf.constant(x_col, dtype=tf.float32)
    t_col_tf = tf.constant(t_col, dtype=tf.float32)

    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            loss_phys = physics_loss(model, x_phys_tf, t_phys_tf, x_col_tf, t_col_tf, alpha, beta, v, D)
            loss_bc = boundary_conditions(model, x_initial_tf, u_initial_tf, x_boundary_tf, u_boundary_tf)
            total_loss = loss_phys + loss_bc

        grads = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 100 == 0:
            print(f'Epoch {epoch+1}, Total Loss: {total_loss.numpy()}, Physics Loss: {loss_phys.numpy()}, BC Loss: {loss_bc.numpy()}')

    print('Training completed.')

def plot_solution(model, x, t, title):
    X, T = np.meshgrid(x, t)
    X_flat = X.flatten()[:, None]
    T_flat = T.flatten()[:, None]
    inputs = np.hstack((X_flat, T_flat))

    U_pred = model.predict(inputs)
    U_pred = np.reshape(U_pred, X.shape)

    fig = go.Figure(data=[go.Surface(x=X, y=T, z=U_pred)])
    fig.update_layout(title=title, autosize=False,
                      scene=dict(
                          xaxis=dict(title='x'),
                          yaxis=dict(title='t'),
                          zaxis=dict(title='u(x,t)'),
                      ))
    pio.show(fig)

# Example usage

# Define the domain and parameters
x_initial = np.linspace(0, 1, 100)[:, None]  # Initial condition points
u_initial = np.sin(np.pi * x_initial)       # Initial condition values

x_boundary = np.array([[0.0], [1.0]])        # Boundary points (example endpoints)
u_boundary = np.zeros_like(x_boundary)      # Boundary condition values (example: zero at boundaries)

x_phys = np.random.uniform(0, 1, (1000, 1))  # Collocation points in the domain
t_phys = np.random.uniform(0, 1, (1000, 1))  # Time points for collocation

x_col = np.random.uniform(0, 1, (1000, 1))   # Collocation points in x-direction
t_col = np.random.uniform(0, 1, (1000, 1))   # Collocation points in t-direction

alpha = 0.5  # Fractional derivative order in time and space
beta = 1.0   # Fractional derivative order in space
v = 1.0      # Advection velocity
D = 0.1      # Diffusion coefficient

# Initialize PINN model
model = PINN()

# Train the PINN
train_PINN(model, x_initial, u_initial, x_boundary, u_boundary, x_phys, t_phys, x_col, t_col, alpha, beta, v, D, epochs=1000)

# Plot the solution using Plotly
x_plot = np.linspace(0, 1, 100)
t_plot = np.linspace(0, 1, 100)
plot_solution(model, x_plot, t_plot, title='Solution of Fractional Advection-Diffusion Equation')


AttributeError: 'SymbolicTensor' object has no attribute 'numpy'